<a href="https://colab.research.google.com/github/BarlaSowmya/AlgoTrading/blob/main/h2h_trial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary packages
!pip install yfinance ta gspread oauth2client scikit-learn --quiet

# Imports
import yfinance as yf
import pandas as pd
import numpy as np
import ta
import logging
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Logging config
logging.basicConfig(level=logging.INFO)

# Define stock list
symbols = ["INFY.NS", "TCS.NS", "RELIANCE.NS"]

# Function: Fetch data and calculate indicators
def fetch_stock_data(symbol, period="6mo", interval="1d"):
    df = yf.Ticker(symbol).history(period=period, interval=interval)
    df.reset_index(inplace=True)
    df['RSI'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()
    df['SMA20'] = ta.trend.SMAIndicator(df['Close'], window=20).sma_indicator()
    df['SMA50'] = ta.trend.SMAIndicator(df['Close'], window=50).sma_indicator()
    return df

# Function: Apply strategy
def apply_strategy(df):
    df['Buy_Signal'] = False
    df['Sell_Signal'] = False
    position = False

    for i in range(1, len(df)):
        rsi = df['RSI'].iloc[i]
        sma20 = df['SMA20'].iloc[i]
        sma50 = df['SMA50'].iloc[i]
        prev_sma20 = df['SMA20'].iloc[i - 1]
        prev_sma50 = df['SMA50'].iloc[i - 1]

        # Required strategy: RSI < 30 AND SMA20 crosses above SMA50
        if not position and rsi < 30 and sma20 > sma50 and prev_sma20 <= prev_sma50:
            df.at[i, 'Buy_Signal'] = True
            position = True
        elif position and rsi > 70:
            df.at[i, 'Sell_Signal'] = True
            position = False

    return df


# Function: Backtesting
def backtest(df):
    buy_price, sell_price, profits = [], [], []
    for i in range(len(df)):
        if df['Buy_Signal'].iloc[i]:
            buy_price.append(df['Close'].iloc[i])
        elif df['Sell_Signal'].iloc[i] and buy_price:
            bp = buy_price.pop(0)
            sp = df['Close'].iloc[i]
            profits.append(sp - bp)
            sell_price.append(sp)
    trades = len(profits)
    wins = sum([1 for p in profits if p > 0])
    win_rate = wins / trades * 100 if trades > 0 else 0
    return {"Total Trades": trades, "Winning Trades": wins, "Win Rate (%)": round(win_rate, 2), "Total Profit": round(sum(profits), 2)}

# Function: ML Model
def ml_prediction(df):
    df.dropna(inplace=True)
    df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    features = df[['RSI', 'SMA20', 'SMA50']].dropna()
    labels = df['Target'].dropna()
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, shuffle=False)
    model = LogisticRegression()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    return round(acc * 100, 2)

def log_trade_log(df, symbol, sheet_client):
    trade_sheet = sheet_client.open("AlgoTradingLog").worksheet("Trade_Log")
    rows = []

    for i in range(len(df)):
        if df['Buy_Signal'].iloc[i]:
            rows.append([str(df['Date'].iloc[i]), symbol, 'BUY', df['Close'].iloc[i]])
        elif df['Sell_Signal'].iloc[i]:
            rows.append([str(df['Date'].iloc[i]), symbol, 'SELL', df['Close'].iloc[i]])

    if trade_sheet.cell(1,1).value is None:
        trade_sheet.append_row(['Date', 'Symbol', 'Action', 'Price'])
    for row in rows:
        trade_sheet.append_row(row)

def log_pnl_log(symbol, result_dict, sheet_client):
    pnl_sheet = sheet_client.open("AlgoTradingLog").worksheet("P&L_Log")
    if pnl_sheet.cell(1,1).value is None:
        headers = ['Symbol', 'Total Trades', 'Winning Trades', 'Win Rate (%)', 'Total Profit', 'ML Accuracy (%)']
        pnl_sheet.append_row(headers)
    pnl_sheet.append_row([symbol] + list(result_dict.values()))

def log_summary(symbols, sheet_client):
    from statistics import mean

    pnl_sheet = sheet_client.open("AlgoTradingLog").worksheet("P&L_Log")
    data = pnl_sheet.get_all_records()
    win_rates = [float(row['Win Rate (%)']) for row in data if row['Symbol'] in symbols]
    total_profit = sum([float(row['Total Profit']) for row in data if row['Symbol'] in symbols])
    ml_acc = [float(row['ML Accuracy (%)']) for row in data if row['Symbol'] in symbols]

    summary_sheet = sheet_client.open("AlgoTradingLog").worksheet("Summary")
    summary_sheet.clear()
    summary_sheet.append_row(['Metric', 'Value'])
    summary_sheet.append_rows([
        ['Average Win Rate (%)', round(mean(win_rates),2)],
        ['Total Net Profit', round(total_profit,2)],
        ['Avg. ML Accuracy (%)', round(mean(ml_acc),2)]
    ])

# Function: Google Sheets Logging
def log_to_gsheet(symbol, df, result_dict):
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name("service_account.json", scope)
    client = gspread.authorize(creds)

    log_trade_log(df, symbol, client)
    log_pnl_log(symbol, result_dict, client)
    log_summary(symbols, client)

# Main runner function
def run_algo():
    for symbol in symbols:
        logging.info(f"Processing {symbol}")
        df = fetch_stock_data(symbol)
        df = apply_strategy(df)
        results = backtest(df)
        accuracy = ml_prediction(df)
        results['ML Accuracy (%)'] = accuracy
        logging.info(results)
        log_to_gsheet(symbol, results)

# Run everything
run_algo()


TypeError: log_to_gsheet() missing 1 required positional argument: 'result_dict'